# Experiment (Get IP address from request)

In [2]:
def get_client_ip(request: gr.Request) -> str:
    """Get the client's IP address from the request."""
    if "cf-connecting-ip" in request.headers:
        return request.headers["cf-connecting-ip"]
    elif "x-forwarded-for" in request.headers:
        return request.headers["x-forwarded-for"]
    else:
        return request.client.host

AttributeError: module 'gradio' has no attribute 'Request'

In [3]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history, request: gr.Request):
        print(get_client_ip(request))
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        time.sleep(2)
        history[-1][1] = bot_message
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch()


AttributeError: module 'gradio' has no attribute 'Blocks'

# State Testing(Text)

## Gradio Interface

In [ ]:
import gradio as gr
from typing import List

def store_message(message: str, history: List[str]):
    output = {
        "Current messages": message,
        "Previous messages": history[::-1]
    }
    history.append(message)
    return output, history

demo = gr.Interface(fn=store_message, 
                    inputs=["textbox", gr.State(value=[])], 
                    outputs=["json", gr.State()])

demo.launch()

## Gradio Block

In [4]:
import gradio as gr
from typing import List

def store_message(message: str, history: List[str]):
    output = {
        "Current messages": message,
        "Previous messages": history[::-1]
    }
    history.append(message)
    return output, history

def clear_history():
    return {}, []

def update(message: str, history: List[str]):
    return store_message(message, history)

with gr.Blocks() as demo:
    state = gr.State(value=[])
    
    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            textbox = gr.Textbox(label="Enter message")
            with gr.Row():
                submit_btn = gr.Button("Submit")
                clear_btn = gr.Button("Clear")
        with gr.Column(scale=1):
            output = gr.JSON(label="Output")

    submit_btn.click(update, inputs=[textbox, state], outputs=[output, state])
    clear_btn.click(clear_history, inputs=[], outputs=[output, state])
    textbox.submit(update, inputs=[textbox, state], outputs=[output, state])

demo.launch()

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [6]:
# Dict version

import gradio as gr
from typing import List, Dict

def store_message(message: str, history: Dict[str, str], count: int):
    output = {
        "Current messages": message,
        "Previous messages": history
    }
    history[str(count)] = message
    count += 1
    return output, history, count

def clear_history():
    return {}, {}, 0

def update(message: str, history: Dict[str, str], count: int):
    return store_message(message, history, count)

with gr.Blocks() as demo:
    state = gr.State(value={})
    count_state = gr.State(value=0)
    
    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            textbox = gr.Textbox(label="Enter message")
            with gr.Row():
                submit_btn = gr.Button("Submit")
                clear_btn = gr.Button("Clear")
        with gr.Column(scale=1):
            output = gr.JSON(label="Output")

    submit_btn.click(update, inputs=[textbox, state, count_state], outputs=[output, state, count_state])
    clear_btn.click(clear_history, inputs=[], outputs=[output, state, count_state])
    textbox.submit(update, inputs=[textbox, state, count_state], outputs=[output, state, count_state])

demo.launch()

AttributeError: module 'gradio' has no attribute 'Blocks'

# State Testing(Chat)

## Gradio ChatInterface

In [ ]:
import gradio as gr
import random

In [ ]:
def yes_man(message, history):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"

gr.ChatInterface(
    yes_man,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7),
    title="Yes Man",
    description="Ask Yes Man any question",
    theme="soft",
    examples=["Hello", "Am I cool?", "Are tomatoes vegetables?"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

## Gradio Block

In [5]:
import gradio as gr

def yes_man(message, history):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"

with gr.Blocks() as demo:
    gr.Markdown("## 👇 Choose any model to chat")

    chatbot = gr.Chatbot(height=300)
    with gr.Row():
        delete_pre = gr.Button("Delete Previous")
        clear = gr.Button("Clear")

    with gr.Row():
        msg = gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7)
        submit_btn = gr.Button("Submit")

    examples = ["Hello", "Am I cool?", "Are tomatoes vegetables?"]
    
    with gr.Row():
        for example in examples:
            gr.Button(example).click(lambda x=example: x, outputs=msg)

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message = yes_man(user_message, history)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            yield history

    def delete_previous(history):
        if history:
            history.pop()
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit_btn.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    delete_pre.click(delete_previous, chatbot, chatbot, queue=False)
    clear.click(lambda: [], None, chatbot, queue=False)

demo.queue()
demo.launch()


AttributeError: module 'gradio' has no attribute 'Blocks'

## Streaming

### Gradio Chatinterface

In [ ]:
import gradio as gr
import time


def yes_man(message):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"
    
def bot(user_message, history):
    bot_message = yes_man(user_message)
    partial_message = ""
    for response in bot_message:
        partial_message += response
        time.sleep(0.02)
        yield partial_message

gr.ChatInterface(
    bot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7),
    title="Yes Man",
    description="Ask Yes Man any question",
    theme="soft",
    examples=["Hello", "Am I cool?", "Are tomatoes vegetables?"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

### Gradio Block

In [ ]:
import gradio as gr
import random
import time

def yes_man(message, history):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"

with gr.Blocks() as demo:
    gr.Markdown("## 👇 Choose any model to chat")

    chatbot = gr.Chatbot(height=300)
    with gr.Row():
        delete_pre = gr.Button("Delete Previous")
        clear = gr.Button("Clear")

    with gr.Row():
        msg = gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7)
        submit_btn = gr.Button("Submit")

    examples = ["Hello", "Am I cool?", "Are tomatoes vegetables?"]
    
    with gr.Row():
        for example in examples:
            gr.Button(example).click(lambda x=example: x, outputs=msg)

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message = yes_man(user_message, history)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

    def delete_previous(history):
        if history:
            history.pop()
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit_btn.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    delete_pre.click(delete_previous, chatbot, chatbot, queue=False)
    clear.click(lambda: [], None, chatbot, queue=False)

demo.queue()
demo.launch()


# With LLM chat completions

## LangChain

### gr.ChatInterface

### gr.Block

# Image

In [30]:
import gradio as gr
import random
import time
import numpy as np

def fake_diffusion(message):
    rng = np.random.default_rng()
    if len(message) < 5:
        image = rng.random(size=(50, 50, 3))
    else:
        image = rng.random(size=(200, 200, 3))
    return image

with gr.Blocks() as demo:
    gr.Markdown("## 👇 Choose any model to chat")

    image = gr.Image(interactive=False)

    with gr.Row():
        msg = gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7)
        submit_btn = gr.Button("Submit")

    msg.submit(fake_diffusion, [msg], [image], queue=False)
    submit_btn.click(fake_diffusion, [msg], [image], queue=False)

demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7901

To create a public link, set `share=True` in `launch()`.


# Markdown

In [1]:
import gradio as gr

def welcome(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    mark = gr.Markdown(
    """
    # Hello World!
    Start typing below to see the output.
    """)
    inp = gr.Textbox(placeholder="What is your name?")
    out = gr.Textbox()
    inp.change(welcome, inp, out)

demo.launch()

AttributeError: module 'gradio' has no attribute 'Blocks'

# Dynamic Layout

In [1]:
import gradio as gr
import requests
from PIL import Image
from io import BytesIO

# Function to call the Stable Diffusion API
def generate_image(prompt_text, prompt_weight, height, width, cfg_scale, clip_guidance_preset, sampler, samples, seed, steps, style_preset):
    api_url = "YOUR_STABLE_DIFFUSION_API_ENDPOINT"
    api_key = "YOUR_API_KEY"  # If the API requires authentication

    # Prepare the payload for the API request
    payload = {
        "text_prompts": [
            {
                "text": prompt_text,
                "weight": prompt_weight
            }
        ],
        "height": height,
        "width": width,
        "cfg_scale": cfg_scale,
        "clip_guidance_preset": clip_guidance_preset,
        "sampler": sampler,
        "samples": samples,
        "seed": seed,
        "steps": steps,
        "style_preset": style_preset
    }

    # Headers for the API request, if needed
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Make the API request
    response = requests.post(api_url, json=payload, headers=headers)

    if response.status_code == 200:
        # Process the API response
        image_data = response.json().get('image')
        image = Image.open(BytesIO(image_data))
        return image
    else:
        return f"Error: {response.status_code}, {response.text}"

In [35]:
interface = gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Prompt Text", lines=2, placeholder="Enter your prompt here..."),
        gr.Slider(label="Prompt Weight", minimum=0.0, maximum=1.0, value=0.5),
        gr.Slider(label="Height", minimum=128, maximum=1024, step=64, value=512),
        gr.Slider(label="Width", minimum=128, maximum=1024, step=64, value=512),
        gr.Slider(label="CFG Scale", minimum=0, maximum=35, step=0.1, value=7),
        gr.Dropdown(label="Clip Guidance Preset", choices=["FAST_BLUE", "FAST_GREEN", "NONE", "SIMPLE", "SLOW", "SLOWER", "SLOWEST"], value="NONE"),
        gr.Dropdown(label="Sampler", choices=["DDIM", "DDPM", "K_DPMPP_2M", "K_DPMPP_2S_ANCESTRAL", "K_DPM_2", "K_DPM_2_ANCESTRAL", "K_EULER", "K_EULER_ANCESTRAL", "K_HEUN", "K_LMS"], value="K_EULER"),
        gr.Slider(label="Samples", minimum=1, maximum=10, step=1, value=1),
        gr.Slider(label="Seed", minimum=0, maximum=4294967295, step=1, value=0),
        gr.Slider(label="Steps", minimum=10, maximum=50, step=1, value=30),
        gr.Dropdown(label="Style Preset", choices=["3d-model", "analog-film", "anime", "cinematic", "comic-book", "digital-art", "enhance", "fantasy-art", "isometric", "line-art", "low-poly", "modeling-compound", "neon-punk", "origami", "photographic", "pixel-art", "tile-texture"], value="photographic")
    ],
    outputs="image",
    title="Stable Diffusion Image Generator",
    description="Generate images using Stable Diffusion. Adjust the parameters and enter a prompt to get started."
)
interface.launch()

Running on local URL:  http://127.0.0.1:7903

To create a public link, set `share=True` in `launch()`.


In [2]:
interface = gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Prompt", lines=2, placeholder="Enter your prompt here..."),
        gr.Textbox(label="Negative Prompt", lines=2, placeholder="Describe what you do not wish to see in the output image..."),
        gr.Dropdown(label="Aspect Ratio", choices=["16:9", "1:1", "21:9", "2:3", "3:2", "4:5", "5:4", "9:16", "9:21"], value="1:1"),
        gr.Slider(label="Height", minimum=128, maximum=1024, step=64, value=512),
        gr.Slider(label="Width", minimum=128, maximum=1024, step=64, value=512),
        gr.Slider(label="CFG Scale", minimum=0, maximum=35, step=0.1, value=7),
        gr.Dropdown(label="Clip Guidance Preset", choices=["FAST_BLUE", "FAST_GREEN", "NONE", "SIMPLE", "SLOW", "SLOWER", "SLOWEST"], value="NONE"),
        gr.Dropdown(label="Sampler", choices=["DDIM", "DDPM", "K_DPMPP_2M", "K_DPMPP_2S_ANCESTRAL", "K_DPM_2", "K_DPM_2_ANCESTRAL", "K_EULER", "K_EULER_ANCESTRAL", "K_HEUN", "K_LMS"], value="K_EULER"),
        gr.Slider(label="Samples", minimum=1, maximum=10, step=1, value=1),
        gr.Slider(label="Seed", minimum=0, maximum=4294967294, step=1, value=0),
        gr.Slider(label="Steps", minimum=10, maximum=50, step=1, value=30),
        gr.Dropdown(label="Style Preset", choices=["3d-model", "analog-film", "anime", "cinematic", "comic-book", "digital-art", "enhance", "fantasy-art", "isometric", "line-art", "low-poly", "modeling-compound", "neon-punk", "origami", "photographic", "pixel-art", "tile-texture"], value="photographic"),
        gr.Dropdown(label="Output Format", choices=["jpeg", "png", "webp"], value="png")
    ],
    outputs="image",
    title="Stable Diffusion Image Generator",
    description="Generate images using Stable Diffusion. Adjust the parameters and enter a prompt to get started."
)
interface.launch()

AttributeError: module 'gradio' has no attribute 'Interface'

In [2]:
def update_layout(selected_layout):
    if selected_layout == "Layout 1":
        return gr.update(visible=True), gr.update(visible=False)
    else:
        return gr.update(visible=False), gr.update(visible=True)
    
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            layout_selector = gr.Dropdown(label="Select Layout", choices=["Layout 1", "Layout 2"], value="Layout 1")
        
        with gr.Column():
            with gr.Row(visible=True) as layout1:
                with gr.Column():
                    prompt_text = gr.Textbox(label="Prompt Text", lines=2, placeholder="Enter your prompt here...")
                    prompt_weight = gr.Slider(label="Prompt Weight", minimum=0.0, maximum=1.0, value=0.5)
                    height = gr.Slider(label="Height", minimum=128, maximum=1024, step=64, value=512)
                    width = gr.Slider(label="Width", minimum=128, maximum=1024, step=64, value=512)
                    cfg_scale = gr.Slider(label="CFG Scale", minimum=0, maximum=35, step=0.1, value=7)
                    clip_guidance_preset = gr.Dropdown(label="Clip Guidance Preset", choices=["FAST_BLUE", "FAST_GREEN", "NONE", "SIMPLE", "SLOW", "SLOWER", "SLOWEST"], value="NONE")
                    sampler = gr.Dropdown(label="Sampler", choices=["DDIM", "DDPM", "K_DPMPP_2M", "K_DPMPP_2S_ANCESTRAL", "K_DPM_2", "K_DPM_2_ANCESTRAL", "K_EULER", "K_EULER_ANCESTRAL", "K_HEUN", "K_LMS"], value="K_EULER")
                    samples = gr.Slider(label="Samples", minimum=1, maximum=10, step=1, value=1)
                    seed = gr.Slider(label="Seed", minimum=0, maximum=4294967295, step=1, value=0)
                    steps = gr.Slider(label="Steps", minimum=10, maximum=50, step=1, value=30)
                    style_preset = gr.Dropdown(label="Style Preset", choices=["3d-model", "analog-film", "anime", "cinematic", "comic-book", "digital-art", "enhance", "fantasy-art", "isometric", "line-art", "low-poly", "modeling-compound", "neon-punk", "origami", "photographic", "pixel-art", "tile-texture"], value="photographic")
                    output_image = gr.Image()

                generate_btn1 = gr.Button("Generate Image")
                generate_btn1.click(
                    generate_image,
                    inputs=[prompt_text, prompt_weight, height, width, cfg_scale, clip_guidance_preset, sampler, samples, seed, steps, style_preset],
                    outputs=output_image
                )

            with gr.Row(visible=False) as layout2:
                with gr.Column():
                    prompt = gr.Textbox(label="Prompt", lines=2, placeholder="Enter your prompt here...")
                    negative_prompt = gr.Textbox(label="Negative Prompt", lines=2, placeholder="Describe what you do not wish to see in the output image...")
                    aspect_ratio = gr.Dropdown(label="Aspect Ratio", choices=["16:9", "1:1", "21:9", "2:3", "3:2", "4:5", "5:4", "9:16", "9:21"], value="1:1")
                    height = gr.Slider(label="Height", minimum=128, maximum=1024, step=64, value=512)
                    width = gr.Slider(label="Width", minimum=128, maximum=1024, step=64, value=512)
                    cfg_scale = gr.Slider(label="CFG Scale", minimum=0, maximum=35, step=0.1, value=7)
                    clip_guidance_preset = gr.Dropdown(label="Clip Guidance Preset", choices=["FAST_BLUE", "FAST_GREEN", "NONE", "SIMPLE", "SLOW", "SLOWER", "SLOWEST"], value="NONE")
                    sampler = gr.Dropdown(label="Sampler", choices=["DDIM", "DDPM", "K_DPMPP_2M", "K_DPMPP_2S_ANCESTRAL", "K_DPM_2", "K_DPM_2_ANCESTRAL", "K_EULER", "K_EULER_ANCESTRAL", "K_HEUN", "K_LMS"], value="K_EULER")
                    samples = gr.Slider(label="Samples", minimum=1, maximum=10, step=1, value=1)
                    seed = gr.Slider(label="Seed", minimum=0, maximum=4294967294, step=1, value=0)
                    steps = gr.Slider(label="Steps", minimum=10, maximum=50, step=1, value=30)
                    style_preset = gr.Dropdown(label="Style Preset", choices=["3d-model", "analog-film", "anime", "cinematic", "comic-book", "digital-art", "enhance", "fantasy-art", "isometric", "line-art", "low-poly", "modeling-compound", "neon-punk", "origami", "photographic", "pixel-art", "tile-texture"], value="photographic")
                    output_format = gr.Dropdown(label="Output Format", choices=["jpeg", "png", "webp"], value="png")
                    output_image = gr.Image()

                generate_btn2 = gr.Button("Generate Image")
                generate_btn2.click(
                    generate_image,
                    inputs=[prompt, negative_prompt, aspect_ratio, height, width, cfg_scale, clip_guidance_preset, sampler, samples, seed, steps, style_preset, negative_prompt, aspect_ratio, output_format],
                    outputs=output_image
                )

    layout_selector.change(update_layout, layout_selector, [layout1, layout2])

if __name__ == "__main__":
    demo.launch()

AttributeError: module 'gradio' has no attribute 'Blocks'